In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/properati_final.csv',error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [8]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

In [9]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


### Busco hiper-parametros con random search

In [14]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

ab = AdaBoostClassifier()

n_estimators =np.arange(40,100,1)
algorithm = ["SAMME", "SAMME.R"]
             
param_grid = {"algorithm": algorithm,"n_estimators":n_estimators}
custom_cv = ShuffleSplit(n_splits=10, test_size=0.2)

random_search = RandomizedSearchCV(ab, param_distributions=param_grid ,cv=custom_cv,n_iter=50,n_jobs=-1)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(random_search.cv_results_['params'])))
print("")
score.report_single(random_search.cv_results_)

RandomizedSearchCV duro 72.32 segundos para 50 candidatos a hyper-parametros.

Model with rank: 1
Mean training score: 0.302 (std: 0.013)
Mean validation score: 0.296 (std: 0.026)
Mean score time: 0.006s
Parameters: {'n_estimators': 41, 'algorithm': 'SAMME'}

Model with rank: 2
Mean training score: 0.306 (std: 0.020)
Mean validation score: 0.296 (std: 0.023)
Mean score time: 0.013s
Parameters: {'n_estimators': 79, 'algorithm': 'SAMME'}

Model with rank: 3
Mean training score: 0.302 (std: 0.017)
Mean validation score: 0.295 (std: 0.024)
Mean score time: 0.007s
Parameters: {'n_estimators': 48, 'algorithm': 'SAMME'}

Model with rank: 4
Mean training score: 0.304 (std: 0.012)
Mean validation score: 0.295 (std: 0.023)
Mean score time: 0.007s
Parameters: {'n_estimators': 52, 'algorithm': 'SAMME'}

Model with rank: 4
Mean training score: 0.310 (std: 0.015)
Mean validation score: 0.295 (std: 0.022)
Mean score time: 0.013s
Parameters: {'n_estimators': 93, 'algorithm': 'SAMME'}

Model with rank:

<IPython.core.display.Javascript object>